In [18]:
%pip install requests beautifulsoup4 pandas selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
from bs4 import BeautifulSoup

# Define the Myntra product URL
URL = "https://www.myntra.com/shirts/mast+%26+harbour/mast--harbour-men-blue--white-slim-fit-striped-casual-shirt/8717979/buy"

# Headers to mimic a real browser visit
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

# Request the webpage
response = requests.get(URL, headers=HEADERS)

# Check if request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extract product name
    product_name = soup.find("h4", class_="product-product").text.strip()
    
    # Extract brand name
    brand_name = soup.find("h3", class_="product-brand").text.strip()
    
    # Extract price
    price = soup.find("span", class_="pdp-price").text.strip()
    
    # Extract discount percentage
    discount = soup.find("span", class_="pdp-discount").text.strip() if soup.find("span", class_="pdp-discount") else "No discount"
    
    # Extract image URL
    image_tag = soup.find("img", class_="image-grid-image")
    image_url = image_tag["src"] if image_tag else "No image found"
    
    # Extract rating
    rating_tag = soup.find("div", class_="pdp-rating")
    rating = rating_tag.text.strip() if rating_tag else "No rating"
    
    # Print extracted details
    print(f"Product Name: {product_name}")
    print(f"Brand: {brand_name}")
    print(f"Price: {price}")
    print(f"Discount: {discount}")
    print(f"Rating: {rating}")
    print(f"Image URL: {image_url}")
    
else:
    print(f"Failed to fetch the webpage. Status code: {response.status_code}")


AttributeError: 'NoneType' object has no attribute 'text'

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Configure headers to mimic browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def scrape_ecom(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract product details (ADJUST SELECTORS BASED ON TARGET SITE)
        products = []
        for item in soup.select('div.product-card'):  # Example selector
            product = {
                'name': item.select_one('h3.product-product').text.strip(),
                'price': item.select_one('span.product-price').text.strip(),
                'image': item.select_one('img.product-image')['src']
            }
            products.append(product)
        return pd.DataFrame(products)
    else:
        print("Failed to retrieve page")
        return None

# Example usage
df = scrape_ecom('https://www.myntra.com/men-tshirts')
df.to_csv('products.csv', index=False)


In [8]:
base_url = "https://www.myntra.com/men-tshirts?p="
for page in range(1, 5):  # Scrape first 4 pages
    scrape_ecom(f"{base_url}{page}")


In [9]:
import time
time.sleep(2)  # Add delay between requests


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def scrape_ecom(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        products = []
        for item in soup.select('div.product-productMetaInfo'):  # Example selector
            product = {
                'name': item.select_one('h3.product-product').text.strip(),
                'price': item.select_one('span.product-discountedPrice').text.strip(),
                'image': item.select_one('img.product-image')['src']
            }
            products.append(product)
        
        # Save and view scraped items
        df = pd.DataFrame(products)
        df.to_csv('scraped_products.csv', index=False)
        print(df.head())  # View first few rows
        
    else:
        print("Failed to retrieve page")

scrape_ecom('https://www.myntra.com/men-clothing')


Empty DataFrame
Columns: []
Index: []


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def scrape_myntra(url):
    # Send HTTP request to fetch page content
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract product details
        products = []
        for item in soup.select('li.product-base'):  # Adjust selector based on inspection
            try:
                product = {
                    'brand': item.select_one('h3.product-brand').text.strip(),
                    'name': item.select_one('h4.product-product').text.strip(),
                    'price': item.select_one('span.product-discountedPrice').text.strip(),
                    'original_price': item.select_one('span.product-strike').text.strip() if item.select_one('span.product-strike') else None,
                    'discount': item.select_one('span.product-discountPercentage').text.strip() if item.select_one('span.product-discountPercentage') else None,
                    'image_url': item.select_one('img.img-responsive')['src'] if item.select_one('img.img-responsive') else None,
                }
                products.append(product)
            except Exception as e:
                print(f"Error scraping product: {e}")
        
        return pd.DataFrame(products)
    else:
        print(f"Failed to fetch page content: {response.status_code}")
        return None

# URL of Myntra's men's clothing page
url = "https://www.myntra.com/men-clothing"

# Scrape data and save it to a CSV file
df = scrape_myntra(url)
if df is not None and not df.empty:
    df.to_csv("myntra_men_clothing.csv", index=False)
    print("Scraped data saved to myntra_men_clothing.csv")
else:
    print("No data scraped.")


No data scraped.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

def scrape_myntra(url):
    driver.get(url)
    time.sleep(3)  # Initial load
    
    # Scroll to load all products (adjust scroll count)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Wait for product elements to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = []
    
    # for item in soup.select('li.product-base'):
    #     print(item.select_one('h3.product-brand').text)  # Check if brand exists
    #     print(item.select_one('h4.product-product'))  # Check if name exists

    # Extract product details
    for item in soup.select('li.product-base'):
        
        soup.prettify()  # View the parsed HTML
        # print(soup.select('li.product-base'))  # Check if product elements are being selected
# Adjust selector
        try:
            brand_element = item.select_one('h3.product-brand').text
            # print(brand_element.text)
            name_element = item.select_one('h4.product-product').text
            price_element = item.select_one('span.product-discountedPrice').text
            ori_price_element = item.select_one('span.product-discountedPrice').text
            image = item.select_one('img.img-responsive')['src']
            print(image)
            # print(price_element)
            
            product = {
                'brand': brand_element if brand_element else "Brand not found",
                'name': name_element if name_element else "Name not found",
                'price': price_element if price_element else "Price not found",
                'original_price': ori_price_element if ori_price_element else "ori_price_element not found",
                'image_url': image if image else "Image not found",
            }
            print(product)
            products.append(product)
            # product = {
            #     'brand': item.select_one('h3.product-brand').text,
            #     'name': item.select_one('h4.product-product').text,
            #     'price': item.select_one('span.product-discountedPrice').text,
            #     # 'original_price': item.select_one('span.product-strike').text if item.select_one('span.product-strike') else None,
            #     # 'image': item.select_one('img.img-responsive')['src'] if item.select_one('img.img-responsive') else None,
            # }
            # products.append(product)
        except Exception as e:
            print(f"Error scraping product: {str(e)}")
            continue
    
    return pd.DataFrame(products)

# Example usage
df = scrape_myntra("https://www.myntra.com/men-clothing")
print(df)
# df.to_csv("myntra_products.csv", index=False)
driver.quit()


https://assets.myntassets.com/dpr_2,q_60,w_210,c_limit,fl_progressive/assets/images/25483098/2024/12/14/b40ee687-855a-4a11-8b99-8ed480d1b9fa1734169559541-HIGHLANDER-Men-Cargo-Trouser-4911734169558855-1.jpg
{'brand': 'HIGHLANDER', 'name': 'Men Mid-Rise Cotton Cargos', 'price': 'Rs. 589', 'original_price': 'Rs. 589', 'image_url': 'https://assets.myntassets.com/dpr_2,q_60,w_210,c_limit,fl_progressive/assets/images/25483098/2024/12/14/b40ee687-855a-4a11-8b99-8ed480d1b9fa1734169559541-HIGHLANDER-Men-Cargo-Trouser-4911734169558855-1.jpg'}
https://assets.myntassets.com/dpr_2,q_60,w_210,c_limit,fl_progressive/assets/images/24546994/2023/8/18/eac84911-3f92-421d-ae18-24525e90b73c1692359658660PrintedBandhgala2PcsSuitInNavyEdward1.jpg
{'brand': 'Blackberrys', 'name': 'Woven Design Two-Piece Suit', 'price': 'Rs. 8816', 'original_price': 'Rs. 8816', 'image_url': 'https://assets.myntassets.com/dpr_2,q_60,w_210,c_limit,fl_progressive/assets/images/24546994/2023/8/18/eac84911-3f92-421d-ae18-24525e90b73

In [37]:
df

,brand,name
0,HIGHLANDER,Men Mid-Rise Cotton Cargos
1,Campus Sutra,Men Solid Cotton Casual Shirt
2,The Souled Store,Men Printed Boxers
3,ONEWAY,Unisex Solid Cotton T-shirt
4,Blackberrys,Woven Design Two-Piece Suit
5,Ramraj,Men Dhotis
6,Roadster,Men Light Fade Skater Jeans
7,Campus Sutra,Men Comfort Pleated Trouser
8,Styli,Men Fit Shirt & Shorts Co-ords
9,Flying Machine,Pure Cotton Relaxed T-shirt


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

def scrape_myntra(url):
    driver.get(url)
    time.sleep(3)  # Initial load
    
    # Scroll multiple times to load all products (adjust scroll count)
    for _ in range(10):  # Increase scroll count for lazy loading
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Extract page source after scrolling
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = []
    
    # Extract product details
    for item in soup.select('li.product-base'):
        try:
            brand_element = item.select_one('h3.product-brand').text.strip()
            name_element = item.select_one('h4.product-product').text.strip()
            price_element = item.select_one('span.product-discountedPrice').text.strip()
            
            # Handle lazy-loaded images with fallback for 'data-src'
            image_element = item.select_one('img.img-responsive')
            image_url = image_element['data-src'] if 'data-src' in image_element.attrs else image_element['src']
            
            product = {
                'brand': brand_element if brand_element else "Brand not found",
                'name': name_element if name_element else "Name not found",
                'price': price_element if price_element else "Price not found",
                'image_url': image_url if image_url else "Image not found",
            }
            products.append(product)
        except Exception as e:
            print(f"Error scraping product: {str(e)}")
    
    return pd.DataFrame(products)

# Example usage
df = scrape_myntra("https://www.myntra.com/men-clothing")
print(df.head())
df.to_csv("myntra_products.csv", index=False)
driver.quit()


Error scraping product: 'NoneType' object has no attribute 'text'
Error scraping product: 'NoneType' object has no attribute 'text'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attr

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

def scrape_myntra(url):
    driver.get(url)
    time.sleep(3)
    
  
    for _ in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = []
 
    for item in soup.select('li.product-base'):
        try:
            brand_element = item.select_one('h3.product-brand').text
            name_element = item.select_one('h4.product-product').text
            price_element = item.select_one('span.product-discountedPrice').text
            
            image_element = item.select_one('img.img-responsive')
            image_url = image_element['data-src'] if 'data-src' in image_element.attrs else image_element['src']
            
            product = {
                'brand': brand_element,
                'name': name_element,
                'price': price_element,
                'image_url': image_url,
            }
            products.append(product)
        except Exception as e:
            print(f"Error scraping product: {e}")
    
    return pd.DataFrame(products)

base_url = "https://www.myntra.com/men-clothing?p="
all_products = []

for page in range(1, 50):  # Scrape first 5 pages
    df_page = scrape_myntra(f"{base_url}{page}")
    all_products.append(df_page)

final_df = pd.concat(all_products)
final_df.to_csv("myntra_all_products.csv", index=False)
driver.quit()


Error scraping product: 'NoneType' object has no attribute 'text'
Error scraping product: 'NoneType' object has no attribute 'text'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attrs'
Error scraping product: 'NoneType' object has no attribute 'attr

KeyboardInterrupt: 

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

def scrape_myntra(url):
    driver.get(url)
    time.sleep(3)  # Initial load
    
    # Scroll to load all products (adjust scroll count)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Wait for product elements to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # products = []
    
    for item in soup.select('li.product-base'):
        print(item.select_one('h3.product-brand').text)  # Check if brand exists
        print(item.select_one('h4.product-product'))  # Check if name exists

    # Extract product details
#     for item in soup.select('li.product-base'):
        
#         print(soup.prettify())  # View the parsed HTML
#         print(soup.select('li.product-base'))  # Check if product elements are being selected
# # Adjust selector
#         try:
#             product = {
#                 'brand': item.select_one('h3.product-brand').text.strip(),
#                 'name': item.select_one('h4.product-product').text.strip(),
#                 'price': item.select_one('span.product-discountedPrice').text.strip(),
#                 'original_price': item.select_one('span.product-strike').text.strip() if item.select_one('span.product-strike') else None,
#                 'image': item.select_one('img.img-responsive')['src'] if item.select_one('img.img-responsive') else None,
#             }
#             products.append(product)
#         except Exception as e:
#             print(f"Error scraping product: {str(e)}")
#             continue
    
#     return pd.DataFrame(products)

# Example usage
df = scrape_myntra("https://www.myntra.com/men-clothing")
df.to_csv("myntra_products.csv", index=False)
driver.quit()


WROGN
<h4 class="product-product">Oversized Pure Cotton T-shirt</h4>
HIGHLANDER
<h4 class="product-product">Men Self Design Cotton Shirt</h4>
The Souled Store
<h4 class="product-product">Men Printed Boxers</h4>
ONEWAY
<h4 class="product-product">Unisex Solid Cotton T-shirt</h4>
DAMENSCH
<h4 class="product-product">DEO-SOFT Deodorizing Trunks</h4>
Ramraj
<h4 class="product-product">Men Dhotis</h4>
Roadster
<h4 class="product-product">Men Light Fade Skater Jeans</h4>
Dennis Lingo
<h4 class="product-product">Men Polo Collar T-shirt</h4>
Styli
<h4 class="product-product">Men Fit Shirt &amp; Shorts Co-ords</h4>
Flying Machine
<h4 class="product-product">Pure Cotton Relaxed T-shirt</h4>
Levis
<h4 class="product-product">Men Solid Trunks</h4>
U.S. Polo Assn.
<h4 class="product-product">Men Relaxed-Fit Shorts</h4>
Mast & Harbour
<h4 class="product-product">Men Slim Fit Casual Shirt</h4>
HIGHLANDER
<h4 class="product-product">Opaque Casual Shirt</h4>
Roadster
<h4 class="product-product">Chambra

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def scrape_myntra(url):
    # Send HTTP request to fetch page content
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract product details
        products = []
        for item in soup.select('li.product-base'):  # Loop through all product elements
            try:
                brand_element = item.select_one('h3.product-brand')
                name_element = item.select_one('h4.product-product')
                
                product = {
                    'brand': brand_element.text.strip() if brand_element else "Brand not found",
                    'name': name_element.text.strip() if name_element else "Name not found",
                }
                products.append(product)
            except Exception as e:
                print(f"Error scraping product: {e}")
        
        return pd.DataFrame(products)
    else:
        print(f"Failed to fetch page content: {response.status_code}")
        return None

# URL of Myntra's men's clothing page
url = "https://www.myntra.com/men-clothing"

# Scrape data and save it to a CSV file
df = scrape_myntra(url)
if df is not None and not df.empty:
    print(df.head())  # Preview first few rows
    df.to_csv("myntra_men_clothing.csv", index=False)
    print("Scraped data saved to myntra_men_clothing.csv")
else:
    print("No data scraped.")


No data scraped.


pip install selenium webdriver-manager pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Selenium

options = webdriver.ChromeOptions()
options.add_argument("--headless") # Run in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(
service=Service(ChromeDriverManager().install()),
options=options
)

def scrape_myntra(url):
driver.get(url)
time.sleep(3) # Initial load

    # Scroll to load all products (adjust scroll count)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Wait for product elements to load
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "product-base"))
    )

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = []

    # Extract product details
    for item in soup.select('li.product-base'):
        try:
            product = {
                'brand': item.select_one('h3.product-brand').text.strip(),
                'name': item.select_one('h4.product-product').text.strip(),
                'price': item.select_one('span.product-discountedPrice').text.strip(),
                'original_price': item.select_one('span.product-strike').text.strip() if item.select_one('span.product-strike') else None,
                'image': item.select_one('img.img-responsive')['src'],
                'rating': item.select_one('div.product-ratingsContainer > span:first-child').text.strip() if item.select_one('div.product-ratingsContainer') else None
            }
            products.append(product)
        except Exception as e:
            print(f"Error scraping product: {str(e)}")
            continue

    return pd.DataFrame(products)

# Example usage

df = scrape_myntra("https://www.myntra.com/men-clothing")
df.to_csv("myntra_products.csv", index=False)
driver.quit()

# pagination:

base_url = "https://www.myntra.com/men-clothing?p="
all_products = []

for page in range(1, 4): # Scrape first 3 pages
print(f"Scraping page {page}")
df = scrape_myntra(f"{base_url}{page}")
all_products.append(df)
time.sleep(5) # Avoid rate limiting

final_df = pd.concat(all_products)
final_df.to_csv("myntra_all_products.csv", index=False)

# proxy rotation

options.add_argument("--proxy-server=123.45.67.89:8080")
